In [3]:
from IPython.display import clear_output 

!pip install webdriver_manager
!pip install selenium
!apt install chromium-chromedriver

clear_output()

In [5]:
from bs4 import BeautifulSoup 

from webdriver_manager.chrome import ChromeDriverManager  # 자동으로 크롬드라이버 (가상드라이버) 파일 다운로드 
from selenium.webdriver.chrome.service import Service  # 다운로드된 크롬드라이버 파일 연결하기 위해 사용 

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options  

import requests

import numpy as np
import pandas as pd
import time
import re 

from IPython.display import Image 

#Word cloud
# from wordcloud import WordCloud
import matplotlib.pyplot as plt
from PIL import Image # 만약 "No module named 'PIL'" 에러가 발생하면 [ pip install Pillow==5.4.1 ] 로 라이브러리를 설치해줍니다.
from nltk.corpus import stopwords 

import warnings
warnings.filterwarnings("ignore") # 불필요한 Warning 메시지를 꺼줍니다.

In [3]:
# 노래 고유 번호(song id) 가져오기 
def get_songid(gnrcode):
    service = Service(executable_path=ChromeDriverManager().install()) 
    # driver = webdriver.Chrome(service = service, chrome_options = options) 
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument("--single-process")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service = service, chrome_options = chrome_options) 

    # 노래 고유번호
    song_id = []
    
    # url의 startindex 변경 -> Page 변경 (10페이지, 장르당 500곡)
    for start_idx in range(1, 452,50): # 1,452,50
        songlist_url = 'https://www.melon.com/genre/song_list.htm?gnrCode=' + gnrcode \
        + '&dtlGnrCode=#params%5BgnrCode%5D=' + gnrcode \
        + '&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj' \
        + '&startIndex=' + str(start_idx)

        print(songlist_url)

        driver.get(songlist_url)
        time.sleep(3)
        web = driver.page_source
        source = BeautifulSoup(web,'html.parser')

        # 노래 고유번호인 Songid를 추출
        for song_info in source.find_all('div',{'class':'ellipsis rank01'}):
            # print('크롤링 중입니다 : ', song_info.find('a')['title'][:-2])
            id = song_info.find('a')['href']
            id = re.findall("\d+", id)[-1]  
            song_id.append(id)
        print('****다음 페이지로 넘어갑니다****')

    # 하나의 장르 크롤링 완료
    print("---------------------------------------------------")
    print("장르 랩 크롤링 완료되었습니다")

    driver.close()
    driver.quit()
    
    return song_id

In [4]:
def songid_to_info(songlist):
    #변수 담을 리스트들 생성
    titles = []
    songID = []
    artists = []
    dates = []
    genres = []
    lyrics = []

    #곡 정보 받아오기
    idx=0
    for songid in songlist:
        url = "https://www.melon.com/song/detail.htm?songId="+ songid

        # songID
        songID.append(songid)


        # print(url)

        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web = requests.get(url,headers=headers).content
        web_page = BeautifulSoup(web, 'html.parser')
        
        # print(web_page)

        #노래 제목 줄바꿈 제거
        title = web_page.find('div',{'class':'song_name'}).get_text().strip().split('\t')[-1]
        if title:
            #title = title.split('(')[0].strip() # (feat.) 등 제거
            titles.append(title)
        else:
            titles.append(' ')
        
        #가수명
        artist = web_page.find('a',{'class':'artist_name'})['title']
        if artist:
            artists.append(artist)
        else:
            artists.append(' ')
            
        #곡 발매일, 장르
        date_genre = web_page.find('dl',{'class':'list'}).get_text()
        if date_genre:
            date = date_genre.split('\n')[4]
            genre = date_genre.split('\n')[6]
            dates.append(date)
            genres.append(genre)
        else:
            dates.append(' ')
            genres.append(' ')

        #가사
        try: 
            lyric = web_page.find('div',{'class':'lyric'}) #.get_text().strip()
            if lyrics:
                lyrics.append(lyric)
            else:
                lyrics.append(' ')
        except: # 성인인증이 필요한 경우 스크래핑 불가 -> 가사 생략
            print('----{} : 성인인증이 필요한 가사입니다----'.format(title))
            lyrics.append(' ')
             

        idx += 1 #다음곡으로 넘어가기
        if idx%100 ==0:
            print(idx, '번째곡 크롤링 완료 :', title)

    info_dict = {'Title':titles, 
                 'SongID': songID,
                 'Artist':artists, 
                 'Date':dates, 
                 'Genre':genres, 
                 'Lyric':lyrics,
                }
    print(len(titles), len(artists), len(dates), len(genres), len(lyrics))
    df_song_info = pd.DataFrame(info_dict)
    return df_song_info

In [16]:
genre_name = '트로트'

# 장르와 장르 id 매칭 
gnr_info = {'발라드':'GN0100', '댄스':'GN0200', '랩_힙합':'GN0300', '알앤비_소울':'GN0400',
           '인디음악':'GN0500', '록_메탈':'GN0600', '트로트':'GN0700','포크_블루스':'GN0800'}

# song_id 가져오기 
song_id = get_songid(gnr_info[genre_name])

# 노래 정보 가져오기 
df_genre_songid_to_info = songid_to_info(song_id) 

# 엑셀 반환 
filename = "{0}_500songs.xlsx".format(genre_name) # 250songs
df_genre_songid_to_info.to_excel(filename, index=False, encoding='utf-8')

https://www.melon.com/genre/song_list.htm?gnrCode=GN0700&dtlGnrCode=#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=1
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0700&dtlGnrCode=#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=51
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0700&dtlGnrCode=#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=101
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0700&dtlGnrCode=#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex=151
****다음 페이지로 넘어갑니다****
https://www.melon.com/genre/song_list.htm?gnrCode=GN0700&dtlGnrCode=#params%5BgnrCode%5D=GN0700&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP

In [7]:
df_genre_songid_to_info

,Title,SongID,Artist,Date,Genre,Lyric
0,해요 (2022),35252996,#안녕,2022.06.07,"발라드, 인디음악",
1,너와 함께,36205780,김민석 (멜로망스),2023.02.26,"발라드, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
2,그라데이션,35361345,10CM,2022.07.03,"인디음악, 포크/블루스","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
3,그댄 행복에 살텐데 (2022),35958308,최유리,2022.12.27,"발라드, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
4,첫사랑,31343875,백아,2018.10.11,"인디음악, 포크/블루스","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
...,...,...,...,...,...,...
495,열기구,32106048,SURL (설),2019.10.18,"인디음악, 록/메탈","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
496,여름별,32741333,정동원,2020.07.05,"발라드, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
497,뜨거워 완전 Totally Hot (Feat. 던밀스),30651619,ZENE THE ZILLA,2017.09.27,"랩/힙합, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."
498,안아줘 (Prod. VAN.C),33374274,오반,2021.03.28,"랩/힙합, 인디음악","[ height:auto; 로 변경시, 확장됨 , \r\n\t\t\t\t\t\t\t..."


In [8]:
df_genre_songid_to_info.Lyric.iloc[1]

<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->
							나 혼자선 무너져 내릴 수도 있던<br/>그날엔 항상 네가 옆에 있어 줘서<br/>어둠이 무서웠던 날들<br/>웃음소리조차 놀림 같던 날에도<br/>버틸만했었지<br/>나를 알아주는 게 그런 한사람 있다는 게<br/>얼마나 행운인지 서서히 알아서<br/>아파도 괜찮을 것 같고<br/>어두운 터널일수록 그곳에서 넌<br/>빛나고 있었지<br/>나의 옆에 있어줘 날 떠나지 말아 줘<br/>모든 아픔들 속에서<br/>홀로 외로워지지 않게<br/>서로의 상처 나누고 있는 이 모습만큼은<br/>나 지킬게 너와 함께<br/>현실 속에서 지키기<br/>쉽지 않은 약속이지만<br/>그래도 넌<br/>나의 옆에 있어줘 날 떠나지 말아 줘<br/>모든 즐거움 속에서<br/>홀로 외로워지지 않게<br/>서로의 기쁨 나누고 있는 이 모습만큼은<br/>나 지킬게 너와 함께<br/>우리 잦은 다툼과<br/>이해하지 못할 숱한 날들이<br/>서롤 비추던 빛나는 눈빛<br/>흐리게 만들 수도 있겠지만<br/>나는 너를 사랑해 우린 서로 사랑해<br/>미래를 약속한 날에 꿈꿨던 우리의 모습<br/>모든 것들이 우리의 눈앞에 이루어지게<br/>노력할게 널 사랑해<br/>
</div>

한 곡만 뽑기

In [15]:
songid = 32508053			
url = "https://www.melon.com/song/detail.htm?songId="+ str(songid)

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web = requests.get(url,headers=headers).content
web_page = BeautifulSoup(web, 'html.parser')

# 제목
title = web_page.find('div',{'class':'song_name'}).get_text().strip() 
title = title.split('\t')[-1] 
# 가수명
artist = web_page.find('a',{'class':'artist_name'})['title'] 
# 발매일, 장르
date_genre = web_page.find('dl',{'class':'list'}).get_text() 
date = date_genre.split('\n')[4] 
genre = date_genre.split('\n')[6] 
#가사
lyric = web_page.find('div',{'class':'lyric'}).get_text().strip() 
# get_text() 제거하면 br태그 같이 나옴
lyric_2 = web_page.find('div',{'class':'lyric'}) #.get_text()#.strip() 
# get_text() 제거하면 br태그 같이 나옴

print(title, artist, date, genre)
print()
print(lyric) 
print(type(lyric))
print()
print(lyric_2) 
print(type(lyric_2))

이제 나만 믿어요 임영웅 2020.04.03 성인가요/트로트

무얼 믿은 걸까 부족했던 내게서나조차 못 믿던 내게 여태 머문 사람무얼 봤던 걸까 가진 것도 없던 내게무작정 내 손을 잡아 날 이끈 사람최고였어그대 눈 속에 비친 내 모습이제는 내게서 그댈 비춰줄게궂은 비가 오면세상 가장 큰 그대 우산이 될게그댄 편히 걸어가요걷다가 지치면내가 그대를 안고 어디든 갈게이제 나만 믿어요나만 두고 가던 나만 스쳐 간 행운이 모여그대가 되어서 내게 와준 거야궂은 비가 오면세상 가장 큰 그대 우산이 될게그댄 편히 걸어가요걷다가 지치면내가 그대를 안고 어디든 갈게이제 나만 믿어요나의 마지막 주인공이 되어다신 누구 앞에서도 그대는 고개 숙이지 마요내가 보지 못했던 홀로 고단했던 시간고맙고 미안해요 사랑해요이 세상은우리를 두고 오랜 장난을 했고우린 속지 않은 거야이제 울지 마요좋을 땐 밤새도록 맘껏 웃어요전부 그대 꺼니까그대는 걱정 말아요이제 나만 믿어요
<class 'str'>

<div class="lyric" id="d_video_summary"><!-- height:auto; 로 변경시, 확장됨 -->
							무얼 믿은 걸까 부족했던 내게서<br/>나조차 못 믿던 내게 여태 머문 사람<br/>무얼 봤던 걸까 가진 것도 없던 내게<br/>무작정 내 손을 잡아 날 이끈 사람<br/>최고였어<br/>그대 눈 속에 비친 내 모습<br/>이제는 내게서 그댈 비춰줄게<br/>궂은 비가 오면<br/>세상 가장 큰 그대 우산이 될게<br/>그댄 편히 걸어가요<br/>걷다가 지치면<br/>내가 그대를 안고 어디든 갈게<br/>이제 나만 믿어요<br/>나만 두고 가던 <br/>나만 스쳐 간 행운이 모여<br/>그대가 되어서 내게 와준 거야<br/>궂은 비가 오면<br/>세상 가장 큰 그대 우산이 될게<br/>그댄 편히 걸어가요<br/>걷다가 지치면<br/>내가 그대를 안고 어디든 갈게<br/>이제 나만 믿어요<br/>나의 마지막 주인공이 되어<br/>다신 누구 앞에서도 <br/>그대는